In [1]:
!pip install larq


     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=d7ea75868702d1a7cd209338aa99d4a36617db88872de1e7766dc240691e9491
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


###Full prec model

In [0]:
import tensorflow as tf
import larq as lq
import numpy as np
import matplotlib.pyplot as plt

In [4]:
num_classes = 10

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_images = train_images.reshape((50000, 32, 32, 3)).astype("float32")
test_images = test_images.reshape((10000, 32, 32, 3)).astype("float32")

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
model = tf.keras.models.Sequential([
    # In the first layer we only quantize the weights and not the input
    tf.keras.layers.Conv2D(128, 3,
                          use_bias=False,
                          input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    tf.keras.layers.Conv2D(128, 3, padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    tf.keras.layers.Conv2D(256, 3, padding="same"),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    tf.keras.layers.Conv2D(256, 3, padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    tf.keras.layers.Conv2D(512, 3, padding="same"),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    tf.keras.layers.Conv2D(512, 3, padding="same"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(1024),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1024),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Activation("softmax")
])
model.summary()

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model.save_weights('/content/gdrive/My Drive/BNN_works/QCIFAR10/DuringTrain/original_model_weights.h5')

In [0]:
model.compile(
    tf.keras.optimizers.Adam(lr=0.01,decay=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [9]:
trained_model = model.fit(
    train_images, 
    train_labels,
    batch_size=50, 
    epochs=50,
    validation_data=(test_images, test_labels),
    shuffle=True
)

Epoch 1/50
1000/1000 [==============================] - 77s 77ms/step - loss: 1.5391 - accuracy: 0.4911 - val_loss: 9.2222 - val_accuracy: 0.3728
Epoch 2/50
1000/1000 [==============================] - 76s 76ms/step - loss: 1.2676 - accuracy: 0.6273 - val_loss: 2.2101 - val_accuracy: 0.5725
Epoch 3/50
1000/1000 [==============================] - 76s 76ms/step - loss: 1.1658 - accuracy: 0.6787 - val_loss: 1.4309 - val_accuracy: 0.6633
Epoch 4/50
1000/1000 [==============================] - 76s 76ms/step - loss: 1.1001 - accuracy: 0.7087 - val_loss: 1.2474 - val_accuracy: 0.6734
Epoch 5/50
1000/1000 [==============================] - 76s 76ms/step - loss: 1.0441 - accuracy: 0.7360 - val_loss: 1.0211 - val_accuracy: 0.6882
Epoch 6/50
1000/1000 [==============================] - 76s 76ms/step - loss: 1.0010 - accuracy: 0.7605 - val_loss: 0.9678 - val_accuracy: 0.6956
Epoch 7/50
1000/1000 [==============================] - 76s 76ms/step - loss: 0.9528 - accuracy: 0.7828 - val_loss: 0.8166 -

KeyboardInterrupt: ignored

###1 - bit model

In [0]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              use_bias=False)

model = tf.keras.models.Sequential([
    # In the first layer we only quantize the weights and not the input
    lq.layers.QuantConv2D(128, 3,
                          kernel_quantizer="ste_sign",
                          kernel_constraint="weight_clip",
                          use_bias=False,
                          input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(256, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(256, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(512, 3, padding="same", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantConv2D(512, 3, padding="same", **kwargs),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Flatten(),

    lq.layers.QuantDense(1024, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantDense(1024, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),

    lq.layers.QuantDense(10, **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999, scale=False),
    tf.keras.layers.Activation("softmax")
])


In [11]:
lq.models.summary(model)

+sequential_1 stats--------------------------------------------------------------------------------------------+
| Layer                   Input prec.            Outputs   # 1-bit  # 32-bit   Memory  1-bit MACs  32-bit MACs |
|                               (bit)                          x 1       x 1     (kB)                          |
+--------------------------------------------------------------------------------------------------------------+
| quant_conv2d                      -  (-1, 30, 30, 128)      3456         0     0.42           0      3110400 |
| batch_normalization_9             -  (-1, 30, 30, 128)         0       256     1.00           0            0 |
| quant_conv2d_1                    1  (-1, 30, 30, 128)    147456         0    18.00   132710400            0 |
| max_pooling2d_3                   -  (-1, 15, 15, 128)         0         0        0           0            0 |
| batch_normalization_10            -  (-1, 15, 15, 128)         0       256     1.00           

In [0]:
model.save_weights('/content/gdrive/My Drive/BNN_works/QCIFAR10/DuringTrain/original_model_weights.h5')

In [0]:
model.compile(
    tf.keras.optimizers.Adam(lr=0.01,decay=0.00003),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [15]:
trained_model = model.fit(
    train_images, 
    train_labels,
    batch_size=50, 
    epochs=100,
    validation_data=(test_images, test_labels),
    shuffle=True
)

Epoch 1/100
1000/1000 [==============================] - 82s 82ms/step - loss: 0.3980 - accuracy: 0.9852 - val_loss: 0.9173 - val_accuracy: 0.7817
Epoch 2/100
1000/1000 [==============================] - 82s 82ms/step - loss: 0.3942 - accuracy: 0.9855 - val_loss: 0.9967 - val_accuracy: 0.7690
Epoch 3/100
1000/1000 [==============================] - 82s 82ms/step - loss: 0.3942 - accuracy: 0.9862 - val_loss: 0.9373 - val_accuracy: 0.7829
Epoch 4/100
1000/1000 [==============================] - 81s 81ms/step - loss: 0.3920 - accuracy: 0.9874 - val_loss: 0.8458 - val_accuracy: 0.8091
Epoch 5/100
1000/1000 [==============================] - 82s 82ms/step - loss: 0.3887 - accuracy: 0.9878 - val_loss: 0.8287 - val_accuracy: 0.8204
Epoch 6/100
1000/1000 [==============================] - 82s 82ms/step - loss: 0.3910 - accuracy: 0.9867 - val_loss: 0.8713 - val_accuracy: 0.8031
Epoch 7/100
1000/1000 [==============================] - ETA: 0s - loss: 0.3912 - accuracy: 0.9870

KeyboardInterrupt: ignored